In [ ]:
# Install library kagglehub untuk memudahkan unduhan via API
# !pip install kagglehub pandas sqlalchemy pymysql

import pandas as pd
import kagglehub
import os
import time
from sqlalchemy import create_engine, text

print("Library API siap!")

Library API siap!



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
db_connection_str = 'mysql+pymysql://root:@localhost/elt_sales_db'
db_connection = create_engine(db_connection_str)

In [39]:
def extract_etl_source2():
    start_time = time.time()
    
    # 1. Download dataset via API (Ekstraksi data mentah) [cite: 61, 62]
    # Dataset: 100000 Sales Records
    print("Mendownload data via Kaggle API...")
    path = kagglehub.dataset_download("okhiriadaveoseghale/100000-sales-records")
    
    # Mencari file CSV di dalam path unduhan
    for file in os.listdir(path):
        if file.endswith(".csv"):
            full_path = os.path.join(path, file)
            break
            
    df = pd.read_csv(full_path)
    
    end_time = time.time()
    execution_time = end_time - start_time
    file_size = os.path.getsize(full_path) / 1024  # Dalam KB
    
    # 2. Membuat log proses extract (Syarat 6.1.5) [cite: 66, 70]
    print("=== [LOG EXTRACT SOURCE 2 - API] ===")
    print(f"Nama Sumber Data : Kaggle API (100000-sales-records)")
    print(f"Jumlah Baris     : {df.shape[0]}")
    print(f"Jumlah Kolom     : {df.shape[1]}")
    print(f"Ukuran Data      : {file_size:.2f} KB")
    print(f"Waktu Eksekusi   : {execution_time:.4f} detik")
    print("====================================")
    
    return df, full_path

df_hasil_ekstraksi = extract_etl_source2()

Mendownload data via Kaggle API...
=== [LOG EXTRACT SOURCE 2 - API] ===
Nama Sumber Data : Kaggle API (100000-sales-records)
Jumlah Baris     : 100000
Jumlah Kolom     : 14
Ukuran Data      : 12191.90 KB
Waktu Eksekusi   : 0.8609 detik


In [40]:
# Cek struktur tabel di database menggunakan SQL
with db_connection.connect() as conn:
    result = conn.execute(text("DESCRIBE raw_sales_api"))
    for row in result:
        print(row)

ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'elt_sales_db.raw_sales_api' doesn't exist")
[SQL: DESCRIBE raw_sales_api]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
# Eksekusi fungsi
df_raw_api, path_original = extract_etl_source2()

# Simpan ke direktori datalake 
if not os.path.exists('datalake'):
    os.makedirs('datalake')

raw_save_path = 'datalake/raw_source2_api.csv'
df_raw_api.to_csv(raw_save_path, index=False)
print(f"✅ Data API berhasil disimpan mentah di: {raw_save_path}")

Mendownload data via Kaggle API...
=== [LOG EXTRACT SOURCE 2 - API] ===
Nama Sumber Data : Kaggle API (100000-sales-records)
Jumlah Baris     : 100000
Jumlah Kolom     : 14
Ukuran Data      : 12191.90 KB
Waktu Eksekusi   : 0.8532 detik
✅ Data API berhasil disimpan mentah di: datalake/raw_source2_api.csv


In [ ]:
table_name_api = 'raw_sales_api'

print(f"Memuat data API ke tabel: {table_name_api}...")

try:
    start_load = time.time()
    
    # Load mentah ke database [cite: 114]
    df_raw_api.to_sql(
        name=table_name_api, 
        con=db_connection, 
        if_exists='replace', 
        index=False, 
        chunksize=1000
    )
    
    end_load = time.time()
    
    # Dokumentasi metadata proses load (Syarat 7.1.3) [cite: 115]
    print("=== [LOG LOAD SOURCE 2] ===")
    print(f"Status           : SUKSES")
    print(f"Tabel Tujuan     : {table_name_api}")
    print(f"Jumlah Baris     : {len(df_raw_api)}")
    print(f"Waktu Eksekusi   : {end_load - start_load:.4f} detik")
    print("===========================")
    
except Exception as e:
    print(f"❌ Gagal memuat data API: {e}")

Memuat data API ke tabel: raw_sales_api...
=== [LOG LOAD SOURCE 2] ===
Status           : SUKSES
Tabel Tujuan     : raw_sales_api
Jumlah Baris     : 100000
Waktu Eksekusi   : 5.9126 detik
